# 03: Final Data Prep

The Animal IDs provided by the Austin Animal Center in each dataset allowed for a convenient way to merge a single dataframe with all information and for animals who have been taken in multiple times, a convenient way to chart their history.

One area of exploration for this project was duration of stay, and merging the Intakes dataset with the Outcomes dataset allowed for this analysis.

Initial imports and file read-ins:

In [1]:
import pandas as pd
import numpy as np

In [2]:
outcomes = pd.read_csv('../data/outcomes_initial.csv').sort_values(['animal_id', 'datetime']).reset_index().drop(columns=['index'])
intakes = pd.read_csv('../data/intakes_initial.csv').sort_values(['animal_id', 'datetime']).reset_index().drop(columns=['index'])

In [3]:
outcomes.head()

,animal_id,name,datetime,outcome_type,outcome_subtype,animal_type,age_upon_outcome,breed,color,is_named,year,month,day,age_range,sex,is_neutered,mix
0,A006100,Scamp,2014-03-08 17:10:00,Return to Owner,Unknown,Dog,6.0,Spinone Italiano Mix,Yellow/White,1,2014,3,Saturday,5 Years-8 Years,Male,Neutered/Spayed,1
1,A006100,Scamp,2014-12-20 16:35:00,Return to Owner,Unknown,Dog,7.0,Spinone Italiano Mix,Yellow/White,1,2014,12,Saturday,5 Years-8 Years,Male,Neutered/Spayed,1
2,A006100,Scamp,2017-12-07 00:00:00,Return to Owner,Unknown,Dog,1.0,Spinone Italiano Mix,Yellow/White,1,2017,12,Thursday,6 Months-2 Years,Male,Neutered/Spayed,1
3,A047759,Oreo,2014-04-07 15:12:00,Transfer,Partner,Dog,1.0,Dachshund,Tricolor,1,2014,4,Monday,6 Months-2 Years,Male,Neutered/Spayed,0
4,A134067,Bandit,2013-11-16 11:54:00,Return to Owner,Unknown,Dog,1.0,Shetland Sheepdog,Brown/White,1,2013,11,Saturday,6 Months-2 Years,Male,Neutered/Spayed,0


In [4]:
intakes.head()

,animal_id,name,datetime,intake_type,intake_condition,animal_type,age_upon_intake,breed,color,is_named,year,month,day,age_range,sex,is_neutered,mix
0,A006100,Scamp,2014-03-07 14:26:00,Public Assist,Normal,Dog,6.0,Spinone Italiano Mix,Yellow/White,1,2014,3,Friday,5 Years-8 Years,Male,Neutered/Spayed,1
1,A006100,Scamp,2014-12-19 10:21:00,Public Assist,Normal,Dog,7.0,Spinone Italiano Mix,Yellow/White,1,2014,12,Friday,5 Years-8 Years,Male,Neutered/Spayed,1
2,A006100,Scamp,2017-12-07 14:07:00,Stray,Normal,Dog,1.0,Spinone Italiano Mix,Yellow/White,1,2017,12,Thursday,6 Months-2 Years,Male,Neutered/Spayed,1
3,A047759,Oreo,2014-04-02 15:55:00,Owner Surrender,Normal,Dog,1.0,Dachshund,Tricolor,1,2014,4,Wednesday,6 Months-2 Years,Male,Neutered/Spayed,0
4,A134067,Bandit,2013-11-16 09:02:00,Public Assist,Injured,Dog,1.0,Shetland Sheepdog,Brown/White,1,2013,11,Saturday,6 Months-2 Years,Male,Neutered/Spayed,0


## I. Additional Columns

Again, many animals go through the shelter system more than once, often with different outcomes each time. Features are generated below to track this history and initially are set to zero for the tallying that follows:

In [5]:
outcomes['prev_adoption'] = [0] * len(outcomes)
outcomes['prev_transfer'] = [0] * len(outcomes)
outcomes['prev_ret_to_owner'] = [0] * len(outcomes)
outcomes['prev_rto_adopt'] = [0] * len(outcomes)
outcomes['prev_disposal'] = [0] * len(outcomes)
outcomes['prev_missing'] = [0] * len(outcomes)
outcomes['prev_relocate'] = [0] * len(outcomes)

Tickers for relevant outcome types are likewise set to zero, and (with the Outcomes dataset sorted by animal and by timestamp respectively) a simple iteration of all the dataset rows allows for a tally of each animal's history based on the respective columns for each of their entries. (For example, an animal who has been returned to his or her owner twice, will on the third time in the shelter system, have `2` in the `prev_ret_to_owner` field.) This allows for tracking whether or not such history influences outcome.

In [6]:
adoptions = 0
transfers = 0
ret_to_owners = 0
rto_adopts = 0
disposals = 0
missings = 0
relocates = 0

for i in range(len(outcomes)- 1):

    if outcomes['animal_id'][i] == outcomes['animal_id'][i + 1]:

        if outcomes['outcome_type'][i] == 'Adoption':
            outcomes['prev_adoption'][i + 1] = adoptions + 1
            adoptions += 1
        elif outcomes['outcome_type'][i] == 'Transfer':
            outcomes['prev_transfer'][i + 1] = transfers + 1
            transfers += 1
        elif outcomes['outcome_type'][i] == 'Return to Owner':
            outcomes['prev_ret_to_owner'][i + 1] = ret_to_owners + 1
            ret_to_owners += 1
        elif outcomes['outcome_type'][i] == 'Rto-Adopt':
            outcomes['prev_rto_adopt'][i + 1] = rto_adopts + 1
            rto_adopts += 1
        elif outcomes['outcome_type'][i] == 'Disposal':
            outcomes['prev_disposal'][i + 1] = disposals + 1
            disposals += 1
        elif outcomes['outcome_type'][i] == 'Missing':
            outcomes['prev_missing'][i + 1] = missings + 1
            missings += 1
        elif outcomes['outcome_type'][i] == 'Relocate':
            outcomes['prev_relocate'][i + 1] = relocates + 1
            relocates += 1
    else:
        adoptions = 0
        transfers = 0
        ret_to_owners = 0
        rto_adopts = 0
        disposals = 0
        missings = 0
        relocates = 0

What follow are a few code cells double-checking that the above iteration worked properly. As information displays correctly, it would seem so.

In [7]:
outcomes[['animal_id', 'outcome_type', 'prev_adoption', 'prev_transfer', 'prev_ret_to_owner', 'prev_rto_adopt', 'prev_disposal', 'prev_missing', 'prev_relocate' ]]

,animal_id,outcome_type,prev_adoption,prev_transfer,prev_ret_to_owner,prev_rto_adopt,prev_disposal,prev_missing,prev_relocate
0,A006100,Return to Owner,0,0,0,0,0,0,0
1,A006100,Return to Owner,0,0,1,0,0,0,0
2,A006100,Return to Owner,0,0,2,0,0,0,0
3,A047759,Transfer,0,0,0,0,0,0,0
4,A134067,Return to Owner,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
129427,A840385,Transfer,0,0,0,0,0,0,0
129428,A840386,Transfer,0,0,0,0,0,0,0
129429,A840402,Euthanasia,0,0,0,0,0,0,0
129430,A840404,Euthanasia,0,0,0,0,0,0,0


In [8]:
outcomes['prev_adoption'].value_counts()

0    121614
1      6660
2       932
3       174
4        38
5        10
6         3
7         1
Name: prev_adoption, dtype: int64

In [9]:
# Checking new features
outcomes.query('animal_id == "A754989"')[['animal_id', 'outcome_type', 'prev_adoption', 'prev_transfer', 'prev_ret_to_owner', 
                                          'prev_rto_adopt', 'prev_disposal', 'prev_missing', 'prev_relocate' ]]

,animal_id,outcome_type,prev_adoption,prev_transfer,prev_ret_to_owner,prev_rto_adopt,prev_disposal,prev_missing,prev_relocate
74019,A754989,Adoption,0,0,0,0,0,0,0
74020,A754989,Adoption,1,0,0,0,0,0,0
74021,A754989,Adoption,2,0,0,0,0,0,0
74022,A754989,Adoption,3,0,0,0,0,0,0
74023,A754989,Adoption,4,0,0,0,0,0,0
74024,A754989,Adoption,5,0,0,0,0,0,0
74025,A754989,Adoption,6,0,0,0,0,0,0
74026,A754989,Adoption,7,0,0,0,0,0,0


In [10]:
outcomes.query('animal_id == "A774102" or animal_id == "A809074"')[['animal_id', 'outcome_type', 'prev_adoption', 'prev_transfer', 'prev_ret_to_owner', 
                                                                    'prev_rto_adopt', 'prev_disposal', 'prev_missing', 'prev_relocate' ]]

,animal_id,outcome_type,prev_adoption,prev_transfer,prev_ret_to_owner,prev_rto_adopt,prev_disposal,prev_missing,prev_relocate
87613,A774102,Transfer,0,0,0,0,0,0,0
87614,A774102,Transfer,0,1,0,0,0,0,0
87615,A774102,Transfer,0,2,0,0,0,0,0
87616,A774102,Return to Owner,0,3,0,0,0,0,0
87617,A774102,Return to Owner,0,0,1,0,0,0,0
87618,A774102,Return to Owner,0,0,2,0,0,0,0
87619,A774102,Return to Owner,0,0,3,0,0,0,0
113857,A809074,Transfer,0,0,0,0,0,0,0
113858,A809074,Transfer,0,1,0,0,0,0,0
113859,A809074,Transfer,0,2,0,0,0,0,0


## II. Combining `Intakes` and `Outcomes` DataFrames

In [11]:
# Converting for rank
intakes['datetime'] = intakes['datetime'].apply(pd.to_datetime)
outcomes['datetime'] = outcomes['datetime'].apply(pd.to_datetime)

Because many animals are entered in the system multiple times, it becomes necessary to track each unique stay, so that each outcome aligns properly for that stay. Below (after a double-checking of the datetime datatypes), a column is created that generates for each animal a unique tracking extension of the ID to mark each unique stay in the shelter. Then a merge is performed so that the two previous datasets are combined into one for future modeling.

In [12]:
intakes['intake_num'] = intakes.groupby(['animal_id'])['datetime'].rank(method='dense', ascending=False)
intakes['tracking_id'] = intakes['animal_id'] + '_' + intakes['intake_num'].astype('int').astype('str')
outcomes['outcome_num'] = outcomes.groupby(['animal_id'])['datetime'].rank(method='dense', ascending=False)
outcomes['tracking_id'] = outcomes['animal_id'] + '_' + outcomes['outcome_num'].astype('int').astype('str')

In [13]:
outcomes.set_index('tracking_id', inplace=True)
intakes.set_index('tracking_id', inplace=True)

full_df = pd.merge(outcomes, intakes, how='inner', 
                  right_index=True, left_index=True, suffixes=['_out', '_in'])

Again, there are multiple duplicate rows between the two initial datasets, and here these are dropped:

In [14]:
full_df.drop(columns=['animal_id_out','breed_out','color_out','intake_num','outcome_num',
                      'animal_type_out','sex_out','is_named_out', 'year_out', 'month_out', 
                      'day_out'], inplace=True)

In [15]:
full_df.head()

,name_out,datetime_out,outcome_type,outcome_subtype,age_upon_outcome,age_range_out,is_neutered_out,mix_out,prev_adoption,prev_transfer,...,breed_in,color_in,is_named_in,year_in,month_in,day_in,age_range_in,sex_in,is_neutered_in,mix_in
tracking_id,,,,,,,,,,,,,,,,,,,,,
A006100_1,Scamp,2017-12-07 00:00:00,Return to Owner,Unknown,1.0,6 Months-2 Years,Neutered/Spayed,1,0,0,...,Spinone Italiano Mix,Yellow/White,1,2017,12,Thursday,6 Months-2 Years,Male,Neutered/Spayed,1
A006100_2,Scamp,2014-12-20 16:35:00,Return to Owner,Unknown,7.0,5 Years-8 Years,Neutered/Spayed,1,0,0,...,Spinone Italiano Mix,Yellow/White,1,2014,12,Friday,5 Years-8 Years,Male,Neutered/Spayed,1
A006100_3,Scamp,2014-03-08 17:10:00,Return to Owner,Unknown,6.0,5 Years-8 Years,Neutered/Spayed,1,0,0,...,Spinone Italiano Mix,Yellow/White,1,2014,3,Friday,5 Years-8 Years,Male,Neutered/Spayed,1
A047759_1,Oreo,2014-04-07 15:12:00,Transfer,Partner,1.0,6 Months-2 Years,Neutered/Spayed,0,0,0,...,Dachshund,Tricolor,1,2014,4,Wednesday,6 Months-2 Years,Male,Neutered/Spayed,0
A134067_1,Bandit,2013-11-16 11:54:00,Return to Owner,Unknown,1.0,6 Months-2 Years,Neutered/Spayed,0,0,0,...,Shetland Sheepdog,Brown/White,1,2013,11,Saturday,6 Months-2 Years,Male,Neutered/Spayed,0


A handful of entries have outcome datetimes that are earlier than intake datetimes (possibly due to midnight being entered as a default outcome time in cases where the real time is unknown. Below, these entries are dropped.

In [16]:
full_df_test = full_df[~(full_df['datetime_out'] < full_df['datetime_in'])]

## III. Standardizing `breed` and `color` Columns

Many breed combinations and many color combinations are included in the original datasets, often with words in different orders and with slashes to separate distinguishing features. Below, the strings in these columns are set to lowercase and alphabetized to eliminate any superfluous duplication of breeds or colors that are actually the same. This is done of course for the purpose of building the strongest model possible later.

In [17]:
full_df_test['breed'] = full_df_test['breed_in'].map(lambda x: ' '.join(set(sorted(x.replace('/', ' ').lower().split(' ')))))
full_df_test['color'] = full_df_test['color_in'].map(lambda x: ' '.join(set(sorted(x.replace('/', ' ').lower().split(' ')))))

In [18]:
full_df_test['breed'].value_counts()

shorthair mix domestic      30788
bull pit mix                 8431
shorthair domestic           8306
retriever mix labrador       6907
chihuahua shorthair mix      6223
                            ...  
clumber spaniel                 1
bull harrier pit                1
shiba inu siberian husky        1
otter                           1
lark mix                        1
Name: breed, Length: 2108, dtype: int64

In [19]:
full_df_test['color'].value_counts()

black white             16795
black                   10756
tabby brown              7226
brown white              6714
white tan                5777
                        ...  
gold chocolate              1
liver tick chocolate        1
orange tricolor             1
tan yellow brindle          1
green gold                  1
Name: color, Length: 373, dtype: int64

In [20]:
full_df_test.head()

,name_out,datetime_out,outcome_type,outcome_subtype,age_upon_outcome,age_range_out,is_neutered_out,mix_out,prev_adoption,prev_transfer,...,is_named_in,year_in,month_in,day_in,age_range_in,sex_in,is_neutered_in,mix_in,breed,color
tracking_id,,,,,,,,,,,,,,,,,,,,,
A006100_2,Scamp,2014-12-20 16:35:00,Return to Owner,Unknown,7.0,5 Years-8 Years,Neutered/Spayed,1,0,0,...,1,2014,12,Friday,5 Years-8 Years,Male,Neutered/Spayed,1,italiano spinone mix,yellow white
A006100_3,Scamp,2014-03-08 17:10:00,Return to Owner,Unknown,6.0,5 Years-8 Years,Neutered/Spayed,1,0,0,...,1,2014,3,Friday,5 Years-8 Years,Male,Neutered/Spayed,1,italiano spinone mix,yellow white
A047759_1,Oreo,2014-04-07 15:12:00,Transfer,Partner,1.0,6 Months-2 Years,Neutered/Spayed,0,0,0,...,1,2014,4,Wednesday,6 Months-2 Years,Male,Neutered/Spayed,0,dachshund,tricolor
A134067_1,Bandit,2013-11-16 11:54:00,Return to Owner,Unknown,1.0,6 Months-2 Years,Neutered/Spayed,0,0,0,...,1,2013,11,Saturday,6 Months-2 Years,Male,Neutered/Spayed,0,sheepdog shetland,brown white
A141142_1,Bettie,2013-11-17 11:40:00,Return to Owner,Unknown,1.0,6 Months-2 Years,Neutered/Spayed,0,0,0,...,1,2013,11,Saturday,6 Months-2 Years,Female,Neutered/Spayed,0,bull pit retriever labrador,black white


## IV. Creating Additional Columns (cont.)

### `days_in_shelter`

Now that the two DataFrames are merged we can collect information on the duration the animal was in the shelter.

In [21]:
full_df_test[['datetime_in','datetime_out']] = full_df_test[['datetime_in','datetime_out']].apply(pd.to_datetime)
full_df_test['days_in_shelter'] = (full_df_test['datetime_out'] - full_df_test['datetime_in']).dt.days

### `adopted`

A column on whether or not an animal is adopted, for a potential binary classification model in later steps.

In [22]:
full_df_test['adopted'] = (full_df_test['outcome_type'] == 'Adoption').map({True: 1, False:0})

## V. Dropping/Renaming Columns

In [23]:
# Dropping
full_df_test.drop(columns = ['animal_id_in','age_range_out', 'outcome_subtype','datetime_in', 'datetime_out',
                             'breed_in','color_in','mix_out','name_out', 'name_in', 'prev_relocate',
                             'age_upon_outcome', 'year_in', 'is_neutered_out'], inplace=True)

In [27]:
# Renaming
full_df_test.rename(columns={'mix_in': 'mix', 'age_range_in':'age_range', 'animal_type_in':'animal_type',
                            'sex_in':'sex', 'adopted_out':'adopted', 'is_neutered_in' : 'is_neutered',
                            'age_range' : 'age_type'}, inplace=True)

In [29]:
# Ordering
full_df_test = full_df_test[['animal_type', 'color', 'breed', 'intake_type', 'outcome_type',
       'intake_condition', 'month_in', 'day_in', 'prev_adoption',
       'prev_transfer', 'prev_ret_to_owner', 'prev_rto_adopt', 'prev_disposal',
       'prev_missing', 'age_upon_intake', 'is_named_in', 'mix', 'sex',
       'is_neutered', 'days_in_shelter', 'age_type', 'adopted']]

## VI. Saving Work

Finally, all is well. A single dataset (clean and optimized for modeling) is written to the `datasets` folder:

In [30]:
full_df_test.to_csv('../data/main.csv', index = False)

# ***Next Notebook*** - [04: Additional EDA]('code/04_Additional_EDA.ipynb')